<a href="https://colab.research.google.com/github/granantuin/Model_vs_data_label/blob/master/all_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,accuracy_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate,GridSearchCV
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

In [0]:
def evaluate(y_test,y_pred):
  index=["E","N","NE","NW","S","SE","SW","W"]
  print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=index, columns=index))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_test, y_pred))," Model Accuracy=32.17%")
  print("****************")
  results= precision_recall_fscore_support(y_test, y_pred, average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print("Average precision =","{:.2%}".format(df["Precision"].mean())," Model precision=21.03%")
  print("Average recall =","{:.2%}".format(df["Recall"].mean())," Model recall=21.71%")
  print(df)
  

In [3]:

from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
y_data=pd.read_excel("/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/y_data.xlsx")
x_data=pd.read_csv("/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/x_data",usecols=range(1,10))

In [0]:
NE=y_data[(y_data["value"]>22.5) & (y_data["value"]<67.5)]
NE["label"]="NE"
E=y_data[(y_data["value"]>67.5) & (y_data["value"]<112.5)]
E["label"]="E"
SE=y_data[(y_data["value"]>112.5) & (y_data["value"]<157.5)]
SE["label"]="SE"
S=y_data[(y_data["value"]>157.5) & (y_data["value"]<202.5)]
S["label"]="S"
SW=y_data[(y_data["value"]>202.5) & (y_data["value"]<247.5)]
SW["label"]="SW"
W=y_data[(y_data["value"]>247.5) & (y_data["value"]<292.5)]
W["label"]="W"
NW=y_data[(y_data["value"]>292.5) & (y_data["value"]<337.5)]
NW["label"]="NW"
N=y_data[(y_data["value"]>337.5) | (y_data["value"]<22.5)]
N["label"]="N"
winds=[NE,N,E,SE,S,SW,W,NW]
y_data=pd.concat(winds)
y_data=y_data.sort_values(by="DATA (Horario UTC)")

**Classifier SVM**

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
evaluate(y_test,y_pred)


    E  N  NE  NW  S  SE  SW  W
E   0  0  12   0  0   0   0  0
N   0  0  12   0  0   0   0  0
NE  0  0  79   0  0   0   0  0
NW  0  0   7   0  0   0   0  0
S   0  0   9   0  0   0   0  0
SE  0  0   4   0  0   0   0  0
SW  0  0  17   0  0   0   0  0
W   0  0  10   0  0   0   0  0
****************
Accuracy= 52.67%  Model Accuracy=32.17%
****************
Average precision = 6.58%  Model precision=21.03%
Average recall = 12.50%  Model recall=21.71%
       Precision  Recall        F1
W_DIR                             
E       0.000000     0.0  0.000000
N       0.000000     0.0  0.000000
NE      0.526667     1.0  0.689956
NW      0.000000     0.0  0.000000
S       0.000000     0.0  0.000000
SE      0.000000     0.0  0.000000
SW      0.000000     0.0  0.000000
W       0.000000     0.0  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Classifier Dummy**

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
y_pred = DummyClassifier(strategy='most_frequent').fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)



    E  N  NE  NW  S  SE  SW  W
E   0  0  12   0  0   0   0  0
N   0  0  12   0  0   0   0  0
NE  0  0  79   0  0   0   0  0
NW  0  0   7   0  0   0   0  0
S   0  0   9   0  0   0   0  0
SE  0  0   4   0  0   0   0  0
SW  0  0  17   0  0   0   0  0
W   0  0  10   0  0   0   0  0
****************
Accuracy= 52.67%  Model Accuracy=32.17%
****************
Average precision = 6.58%  Model precision=21.03%
Average recall = 12.50%  Model recall=21.71%
       Precision  Recall        F1
W_DIR                             
E       0.000000     0.0  0.000000
N       0.000000     0.0  0.000000
NE      0.526667     1.0  0.689956
NW      0.000000     0.0  0.000000
S       0.000000     0.0  0.000000
SE      0.000000     0.0  0.000000
SW      0.000000     0.0  0.000000
W       0.000000     0.0  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Classifier Randomforest**

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
y_pred = RandomForestClassifier(n_estimators=20).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

    E  N  NE  NW  S  SE  SW  W
E   2  0  10   0  0   0   0  0
N   0  1  10   0  0   0   0  1
NE  6  2  64   2  0   0   1  4
NW  0  0   6   0  0   0   1  0
S   0  1   5   1  0   0   2  0
SE  0  1   3   0  0   0   0  0
SW  0  2  14   0  1   0   0  0
W   0  1   6   0  1   0   2  0
****************
Accuracy= 44.67%  Model Accuracy=32.17%
****************
Average precision = 11.47%  Model precision=21.03%
Average recall = 13.25%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.250000  0.166667  0.200000
N       0.125000  0.083333  0.100000
NE      0.542373  0.810127  0.649746
NW      0.000000  0.000000  0.000000
S       0.000000  0.000000  0.000000
SE      0.000000  0.000000  0.000000
SW      0.000000  0.000000  0.000000
W       0.000000  0.000000  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Oversample minority class**

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
X = pd.concat([x_train, y_train], axis=1)
not_NE = X[X["label"]!="NE"]
NE = X[X["label"]=="NE"]
not_NE_upsampled = resample(not_NE,
                          replace=True, # sample with replacement
                          n_samples=len(not_NE), # match number in majority class
                          random_state=27) # reproducible results
upsampled = pd.concat([NE, not_NE_upsampled])
y_train=upsampled["label"]
x_train=upsampled.drop("label",axis=1)
y_pred = RandomForestClassifier(n_estimators=20).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)


    E  N  NE  NW  S  SE  SW  W
E   2  0  10   0  0   0   0  0
N   0  1   9   1  0   0   0  1
NE  3  4  70   1  0   0   0  1
NW  0  0   6   0  0   0   1  0
S   0  1   6   0  0   1   1  0
SE  0  1   3   0  0   0   0  0
SW  1  1  13   0  1   0   0  1
W   0  1   5   0  2   1   1  0
****************
Accuracy= 48.67%  Model Accuracy=32.17%
****************
Average precision = 12.73%  Model precision=21.03%
Average recall = 14.20%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.333333  0.166667  0.222222
N       0.111111  0.083333  0.095238
NE      0.573770  0.886076  0.696517
NW      0.000000  0.000000  0.000000
S       0.000000  0.000000  0.000000
SE      0.000000  0.000000  0.000000
SW      0.000000  0.000000  0.000000
W       0.000000  0.000000  0.000000


**Generate synthetic samples**

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
print("Original train samples=",x_train.shape[0])
#y_pred = RandomForestClassifier(n_estimators=15).fit(x_train, y_train).predict(x_test)
#evaluate(y_test["label"],y_pred)

sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
print("Synthetic train samples=",x_train.shape[0])
print("****Randomforest****")
y_pred = RandomForestClassifier(n_estimators=15).fit(x_train, y_train).predict(x_test)
evaluate(y_test["label"],y_pred)
print("****SVM****")
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
evaluate(y_test["label"],y_pred)


Original train samples= 450
Synthetic train samples= 2024
****Randomforest****
     E   N  NE  NW  S  SE  SW  W
E    5   2   2   1  0   0   0  0
N    1   1   4   0  0   1   1  2
NE  10  12  38   3  4   4   7  3
NW   0   1   1   2  1   0   1  1
S    1   1   4   0  3   0   3  0
SE   0   1   3   0  0   0   1  1
SW   1   2   3   0  4   0   1  1
W    0   1   5   0  0   0   2  4
****************
Accuracy= 36.00%  Model Accuracy=32.17%
****************
Average precision = 24.22%  Model precision=21.03%
Average recall = 25.27%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.277778  0.500000  0.357143
N       0.047619  0.100000  0.064516
NE      0.633333  0.469136  0.539007
NW      0.333333  0.285714  0.307692
S       0.250000  0.250000  0.250000
SE      0.000000  0.000000  0.000000
SW      0.062500  0.083333  0.071429
W       0.333333  0.333333  0.333333
****SVM****


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


    E  N  NE  NW  S  SE  SW  W
E   0  0  10   0  0   0   0  0
N   0  0   9   0  0   1   0  0
NE  0  0  80   0  0   1   0  0
NW  0  0   7   0  0   0   0  0
S   0  0  11   0  0   0   1  0
SE  0  0   5   0  0   0   1  0
SW  0  0  12   0  0   0   0  0
W   0  0  12   0  0   0   0  0
****************
Accuracy= 53.33%  Model Accuracy=32.17%
****************
Average precision = 6.85%  Model precision=21.03%
Average recall = 12.35%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.000000  0.000000  0.000000
N       0.000000  0.000000  0.000000
NE      0.547945  0.987654  0.704846
NW      0.000000  0.000000  0.000000
S       0.000000  0.000000  0.000000
SE      0.000000  0.000000  0.000000
SW      0.000000  0.000000  0.000000
W       0.000000  0.000000  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Cross validation K_folds**

In [16]:
clf=RandomForestClassifier(n_estimators=15)
scoring = ['precision_macro', 'recall_macro','f1_macro',"accuracy"]
scores = cross_validate(clf, x_data, y_data["label"], scoring=scoring,
                        cv=5, return_train_score=False)
print("Accuracy: {:.2%} (+/- {:.2%})" .format (scores["test_accuracy"].mean(), scores["test_accuracy"].std() * 2))
print("Recall: {:.2%} (+/- {:.2%})" .format (scores["test_recall_macro"].mean(), scores["test_recall_macro"].std() * 2))
print("Precision: {:.2%} (+/- {:.2%})" .format (scores["test_precision_macro"].mean(), scores["test_precision_macro"].std() * 2))
print("f1: {:.2%} (+/-{:.2%} )".format (scores["test_f1_macro"].mean(), scores["test_f1_macro"].std() * 2))

Accuracy: 41.56% (+/- 25.94%)
Recall: 13.43% (+/- 4.45%)
Precision: 12.69% (+/- 10.01%)
f1: 12.44% (+/-6.21% )


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Tuning SVM**

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
clf = GridSearchCV(svm.SVC(), tuned_parameters[1],scoring="f1_macro", cv=5).fit(x_train, y_train["label"])
                    
print("Best parameters=",clf.best_params_)
print("Best F1=",clf.best_score_)


KeyboardInterrupt: ignored